In [12]:
# ===============================
# 1️⃣ IMPORTS
# ===============================
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

# ===============================
# 2️⃣ LOAD DATA
# ===============================
df = pd.read_csv("final_df_binary.csv", parse_dates=['date'])
df = df.sort_values(['section_id', 'date']).reset_index(drop=True)

# ===============================
# 3️⃣ TRAIN/TEST SPLIT
# ===============================
train_df = df[df['date'].dt.year <= 2023].copy()
test_df = df[df['date'].dt.year == 2024].copy()

# ===============================
# 4️⃣ SECTION LIST
# ===============================
sections = df['section_id'].unique()

# ===============================
# 5️⃣ FUNCTION TO TUNE PROPHET HYPERPARAMS WITH OPTUNA
# ===============================
def tune_prophet(train_section_df):
    section_id = train_section_df['section_id'].iloc[0] if 'section_id' in train_section_df.columns else 'Unknown'
    print(f"[Optuna] Tuning section: {section_id}")
    def objective(trial):
        # Ensure correct columns for Prophet
        df = train_section_df.rename(columns={'date': 'ds', 'total_task_time_minutes': 'y'})
        changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.0001, 0.5, log=True)
        seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 31.0, log=True)
        holidays_prior_scale = trial.suggest_float('holidays_prior_scale', 0.01, 31.0, log=True)
        
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays_prior_scale=holidays_prior_scale
        )
        # Add regressors
        model.add_regressor('is_weekend')
        model.add_regressor('is_holiday')
        
        model.fit(df[['ds','y','is_weekend','is_holiday']])
        
        # Cross-validation on last 90 days, initial window increased to 730 days
        df_cv = cross_validation(model, horizon='90 days', period='30 days', initial='730 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=0.1)
        return df_p['mae'].values[0]  # Minimize MAE
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=40, show_progress_bar=True)
    return study.best_params

# ===============================
# 6️⃣ SECTION-WISE MODELING
# ===============================
results_dict = {}
stats_dict = {}

for sec in sections:
    print(f"[Main Loop] Now tuning section: {sec}")
    # Prepare data
    train_sec = train_df[train_df['section_id']==sec].copy()
    test_sec = test_df[test_df['section_id']==sec].copy()
    # Prophet format (rename columns before passing to tune_prophet)
    train_sec = train_sec.rename(columns={'date':'ds', 'total_task_time_minutes':'y'})
    test_sec = test_sec.rename(columns={'date':'ds', 'total_task_time_minutes':'y'})

    # Tune hyperparameters
    best_params = tune_prophet(train_sec)
    print(f"{sec} best params:", best_params)
    
    # Fit final model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=best_params['changepoint_prior_scale'],
        seasonality_prior_scale=best_params['seasonality_prior_scale'],
        holidays_prior_scale=best_params['holidays_prior_scale']
    )
    model.add_regressor('is_weekend')
    model.add_regressor('is_holiday')
    model.fit(train_sec[['ds','y','is_weekend','is_holiday']])
    
    # Predict 2024
    forecast = model.predict(test_sec[['ds','is_weekend','is_holiday']])
    test_sec['predicted_task_minutes'] = forecast['yhat'].values
    test_sec['error'] = test_sec['y'] - test_sec['predicted_task_minutes']
    
    # Save section DataFrame
    results_dict[sec] = test_sec[['ds','is_weekend','is_holiday','y','predicted_task_minutes','error']].rename(
        columns={'ds':'date','y':'actual_task_minutes'}
    )
    
    # Section-wise stats
    train_stats = {
        'mean_minutes': train_sec['y'].mean(),
        'mode_minutes': train_sec['y'].mode()[0],
        'max_minutes': train_sec['y'].max(),
        'median_minutes': train_sec['y'].median(),
        'abs_error_mean': np.nan  # placeholder
    }
    test_stats = {
        'mean_minutes': test_sec['y'].mean(),
        'mode_minutes': test_sec['y'].mode()[0],
        'max_minutes': test_sec['y'].max(),
        'median_minutes': test_sec['y'].median(),
        'abs_error_mean': test_sec['error'].abs().mean()
    }
    stats_dict[sec] = {'train': train_stats, 'test': test_stats}

# ===============================
# 7️⃣ VISUALIZATIONS SECTION-WISE
# ===============================
for sec in sections:
    df_plot = results_dict[sec]
    plt.figure(figsize=(10,6))
    sns.scatterplot(x='actual_task_minutes', y='predicted_task_minutes', alpha=0.7)
    plt.plot([0, df_plot['actual_task_minutes'].max()],
             [0, df_plot['actual_task_minutes'].max()],
             'r--', linewidth=2)
    plt.title(f"{sec} Actual vs Predicted Task Minutes (2024)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.show()
    
    plt.figure(figsize=(12,5))
    plt.plot(df_plot['date'], df_plot['actual_task_minutes'], label='Actual')
    plt.plot(df_plot['date'], df_plot['predicted_task_minutes'], label='Predicted')
    plt.title(f"{sec} Time Series Plot 2024")
    plt.xlabel("Date")
    plt.ylabel("Task Minutes")
    plt.legend()
    plt.show()

# ===============================
# 8️⃣ RESULTS
# ===============================
# results_dict contains six DataFrames, one per section
# stats_dict contains section-wise stats for train and test


[I 2025-08-22 18:55:10,563] A new study created in memory with name: no-name-481e39d5-821b-4c94-8723-a77751f6cb56


[Main Loop] Now tuning section: SEC-001
[Optuna] Tuning section: SEC-001


  0%|          | 0/40 [00:00<?, ?it/s]

18:55:10 - cmdstanpy - INFO - Chain [1] start processing
18:55:10 - cmdstanpy - INFO - Chain [1] done processing
18:55:10 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
18:55:10 - cmdstanpy - INFO - Chain [1] done processing
18:55:10 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
18:55:10 - cmdstanpy - INFO - Chain [1] start processing
18:55:10 - cmdstanpy - INFO - Chain [1] start processing
18:55:12 - cmdstanpy - INFO - Chain [1] done processing
18:55:12 - cmdstanpy - INFO - Chain [1] done processing


[I 2025-08-22 19:01:54,909] Trial 0 finished with value: 376.3318966367167 and parameters: {'changepoint_prior_scale': 0.0001334040405545397, 'seasonality_prior_scale': 14.470539260154986, 'holidays_prior_scale': 4.814086154301914}. Best is trial 0 with value: 376.3318966367167.


...

19:45:00 - cmdstanpy - INFO - Chain [1] start processing
19:45:00 - cmdstanpy - INFO - Chain [1] done processing
19:45:00 - cmdstanpy - INFO - Chain [1] done processing


[I 2025-08-22 19:45:08,735] Trial 19 finished with value: 800.9389176831418 and parameters: {'changepoint_prior_scale': 0.01553202148725182, 'seasonality_prior_scale': 8.946461500372855, 'holidays_prior_scale': 3.397057345095677}. Best is trial 12 with value: 787.0550895529786.


19:45:09 - cmdstanpy - INFO - Chain [1] start processing
19:45:09 - cmdstanpy - INFO - Chain [1] done processing
19:45:09 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:45:10 - cmdstanpy - INFO - Chain [1] start processing
19:45:09 - cmdstanpy - INFO - Chain [1] done processing
19:45:09 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:45:10 - cmdstanpy - INFO - Chain [1] start processing
19:45:13 - cmdstanpy - INFO - Chain [1] done processing
19:45:13 - cmdstanpy - INFO - Chain [1] done processing


[W 2025-08-22 19:45:20,710] Trial 20 failed with parameters: {'changepoint_prior_scale': 0.0002318341318773229, 'seasonality_prior_scale': 0.013001104471164843, 'holidays_prior_scale': 0.7869197850852833} because of the following error: UnboundLocalError("cannot access local variable 'fd_out' where it is not associated with a value").
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "d:\tech_triathlon_datathon\venv\Lib\site-packages\prophet\models.py", line 126, in fit
    self.stan_fit = self.model.optimize(**args)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tech_triathlon_datathon\venv\Lib\site-packages\cmdstanpy\model.py", line 659, in optimize
    raise RuntimeError(msg)
RuntimeError: Error during optimization! Command 'D:\tech_triathlon_datathon\venv\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=53441 data file=C:\Users\isuru\AppData\Local\Temp\tmpbt4uiay4\yeaiu9jh.json init=C:\Users\isuru\AppData\Local

UnboundLocalError: cannot access local variable 'fd_out' where it is not associated with a value